# Data from prodigy instance

The AWS EC2 instance where we used to run Prodigy is at prodigy.bdrc.io 
The instance has all the data before we moved to stt.pecha.tools
Only the Quality Control team has worked on the prodigy instance recently to review all task on prodigy.

To load data form prodigy, log into the server
> ssh spsither@prodigy.bdrc.io

Run the following script to export all the data from prodigy to jsonl files
>/home/spsither/staging/export_all.sh

Create a tar with all the jsonl files
>tar -cJvf staging.tar staging/*.jsonl

---

Task that has been transcribed or reviewed but not finalised by the final reviewer are imported in stt.pecha.tools in `prodigy` group. When using data from staging.tar use only the finale_df all others will be in 01_stt_pecha_tools.

In [ ]:
# bring the tar file from the server to the local machine
! scp spsither@prodigy.bdrc.io:/home/spsither/staging.tar data/

In [ ]:
# extract the tar file
! tar -xf data/staging.tar -C data/

In [ ]:
import pandas as pd
def read_jsonl(filename):
    df = pd.read_json(filename,lines=True)
    df = df.reset_index(drop=True)
    return df

In [ ]:
import os

jsonl_directory = "data/staging/"

files = os.listdir(jsonl_directory)

pattern = "jsonl"
files_matching_pattern = [file for file in files if file.endswith(pattern)]

In [ ]:
#all reviewed files 
reviewed_jsonl_files = []
for file in files_matching_pattern:
    if 'review' in file and not 'stt_second_review' in file:
        reviewed_jsonl_files.append(file)

In [ ]:
#all un-reviewed files
transcribed_jsonl_files = []
for file in files_matching_pattern:
    if 'review' not in file and not 'stt_second_review' in file:
        transcribed_jsonl_files.append(file)

In [ ]:
! pip install tqdm
! pip install ipywidgets

In [ ]:
#concatenate all the transcribed dataFrames
temp = pd.DataFrame([])
from tqdm.auto import tqdm
for file in tqdm(transcribed_jsonl_files):
    df = read_jsonl(f"{jsonl_directory}/{file}")
    temp = pd.concat([temp,df],axis=0)
    transcribed_df = temp.reset_index(drop=True)

In [ ]:
#concatenate all the reviewed dataFrames
from tqdm.auto import tqdm
temp = pd.DataFrame([])
for file in tqdm(reviewed_jsonl_files):
    df = read_jsonl(f"{jsonl_directory}/{file}")
    temp = pd.concat([temp,df],axis=0)
    reviewed_df = temp.reset_index(drop=True)

In [ ]:
finale_df = read_jsonl(f"{jsonl_directory}/stt_second_review.jsonl")

In [ ]:
transcribed_df['id'].fillna(transcribed_df['text'], inplace=True)
reviewed_df['id'].fillna(reviewed_df['text'], inplace=True)
finale_df['id'].fillna(finale_df['text'], inplace=True)

In [ ]:
audio_extensions = ['.mp3', '.wav', '.MP3', '.WAV']

In [ ]:
for audio_extension in audio_extensions:
    transcribed_df['id'] = transcribed_df['id'].str.replace(audio_extension, '')
    reviewed_df['id'] = reviewed_df['id'].str.replace(audio_extension, '')
    finale_df['id'] = finale_df['id'].str.replace(audio_extension, '')

In [ ]:
len(transcribed_df),len(reviewed_df),len(finale_df)

In [ ]:
len(set(transcribed_df['id'])), len(set(reviewed_df['id'])), len(set(finale_df['id']))

In [ ]:
100 - len(set(transcribed_df['id'])) / len(transcribed_df) * 100

In [ ]:
transcribed_df.shape[0], reviewed_df.shape[0], finale_df.shape[0]

In [ ]:
transcribed_df.drop_duplicates(subset='id', keep="first", inplace=True)
reviewed_df.drop_duplicates(subset='id', keep="first", inplace=True)
finale_df.drop_duplicates(subset='id', keep="first", inplace=True)

In [ ]:
transcribed_df.shape[0], reviewed_df.shape[0], finale_df.shape[0]

In [ ]:
len(transcribed_df),len(reviewed_df),len(finale_df)

In [ ]:
transcribed_df = transcribed_df[transcribed_df['answer'] == 'accept']
reviewed_df = reviewed_df[reviewed_df['answer'] == 'accept']
finale_df = finale_df[finale_df['answer'] == 'accept']

In [ ]:
len(transcribed_df),len(reviewed_df),len(finale_df)

In [ ]:
# leakage check
len(transcribed_df) - len(finale_df), len(transcribed_df) - len(reviewed_df)

In [ ]:
# left side intersection
intersection = reviewed_df.merge(finale_df, how='left', on='id')

In [ ]:
intersection['grade'] = 1 # 1 means the task is transcribed only

In [ ]:
intersection.loc[~intersection['transcript_x'].isna(), 'grade'] = 2 # 2 means the task is reviewed

In [ ]:
intersection.loc[~intersection['transcript_y'].isna(), 'grade'] = 3 # 3 means the task is reviewed twice by the qc team

In [ ]:
intersection['transcript_y'].fillna(intersection['transcript_x'], inplace=True) # overwrite the transcribed text with the reviewed text

In [ ]:
intersection = intersection[~intersection['transcript_y'].isna()]

In [ ]:
intersection = intersection[['transcript_y', 'id', 'grade']]

In [ ]:
intersection['url'] = 'https://d38pmlk0v88drf.cloudfront.net/wav/' + intersection['id'] + '.wav'

In [ ]:
intersection.shape

In [ ]:
i = 11001
intersection.loc[i]['transcript_y'], intersection.loc[i,'url']

In [ ]:
intersection['dept'] = intersection['id'].str[:6]

In [ ]:
intersection.groupby('dept').size()

In [ ]:
intersection.rename(columns={"id": "file_name", "transcript_y": "uni", "url": "url", "dept": "dept", "grade": "grade"}, inplace=True)

In [ ]:
last_intersection = intersection[['file_name', 'uni', 'url', 'dept', 'grade']]
last_intersection.head()

In [ ]:
last_intersection.to_csv('02_prodigy.tsv', sep='\t', index=False)

In [ ]:
last_intersection[last_intersection['file_name'] == 'STT_AB00001_0029_150584_to_151748']